In [1]:
from KGGraph import load_toxcast_dataset

smiles_list, mols_list, labels = load_toxcast_dataset("dataset/classification/toxcast/raw/toxcast.csv")

In [2]:
labels

array([[-1., -1.,  0., ..., -1., -1., -1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])